# Import all possessions of a season using PBP API

In [1]:
import pandas as pd
import numpy as np
# import time
# import datetime
from IPython.display import clear_output
from collections import Counter

## Import all the Game IDs in a season

In [2]:
from pbpstats.client import Client

settings = {
    "Games": {"source": "web", "data_provider": "data_nba"},
     "dir": "C:/Users/spannala/Documents/Sra_coding/NBA/pbpdata",
}
client = Client(settings)
# ID of all games for 2020-21 Season
# season = client.Season("nba", "2019-20", "Regular Season")
season = client.Season("wnba", "2019", "Regular Season")
games_id = []
k = 0
for final_game in season.games.final_games:
    k += 1
    clear_output(wait=True)
    print(k)
    games_id.append(final_game['game_id'])

204


## Import all Game data objects in a season

In [3]:
settings = {
    "Boxscore": {"source": "file", "data_provider": "data_nba"},
    "Possessions": {"source": "file", "data_provider": "data_nba"},
    "dir": "C:/Users/spannala/Documents/Sra_coding/NBA/pbpdata"
}
client = Client(settings)
games_list = []
bad_games_list = []
k = 0
for gameid in games_id:
    clear_output(wait=True)
    k += 1
    print(k)
    try:
        games_list.append(client.Game(gameid))
    except:
        print(gameid)
        bad_games_list.append(gameid)
        continue

204
1021900204


Number of bad games

In [4]:
len(bad_games_list)

3

# For loop for generating lists of all revelant data

In [5]:
from pbpstats.resources.enhanced_pbp import Rebound
from pbpstats.resources.enhanced_pbp import Turnover
from pbpstats.resources.enhanced_pbp import FieldGoal
from pbpstats.resources.enhanced_pbp import FreeThrow
pos = []
posdec = []
points = []
off1, off2, off3, off4, off5 = [],[],[],[],[]
def1, def2, def3, def4, def5 = [],[],[],[],[]
for game in games_list:
    hmscr = []
    rdscr = []
    for possession in game.possessions.items:
        for possession_event in possession.events:
            if possession_event.count_as_possession:
                if possession_event.description == 'Game End':
                    continue
                pos.append(possession_event)
                posdec.append(possession_event.description)
                hmscr.append(possession_event.home_score)
                rdscr.append(possession_event.away_score)
                off_id =  possession_event.get_offense_team_id()
                teams = list(possession_event.current_players.keys())
                for te in teams:
                    if te == off_id:
                        off1.append(possession_event.current_players[te][0])
                        off2.append(possession_event.current_players[te][1])
                        off3.append(possession_event.current_players[te][2])
                        off4.append(possession_event.current_players[te][3])
                        off5.append(possession_event.current_players[te][4])
                    else:
                        def1.append(possession_event.current_players[te][0])
                        def2.append(possession_event.current_players[te][1])
                        def3.append(possession_event.current_players[te][2])
                        def4.append(possession_event.current_players[te][3])
                        def5.append(possession_event.current_players[te][4])
    hm_pts = []
    rd_pts = []
    for i in range(len(hmscr)-1):
        hm_pts.append(hmscr[i+1]-hmscr[i])
        rd_pts.append(rdscr[i+1]-rdscr[i])
    pts = []
    pts.append(max(hmscr[0],hmscr[0]))
    for i in range(len(hmscr)-1):
        pts.append(max(hm_pts[i],rd_pts[i]))
    points.extend(pts)

## Save DataFrame to .csv file

In [6]:
# data1 = pd.DataFrame({'Possession':posdec,'Home':hmscr,'Road':rdscr,'Points':pts})
data = pd.DataFrame({'off1':off1, 'off2':off2, 'off3':off3, 'off4':off4, 'off5':off5, 'def1':def1, 'def2':def2, 'def3':def3, 'def4':def4, 'def5':def5, 'Points':points})
data.to_csv ('WNBA_rapm_possessions_2019.csv',index=False)